## This notebook provides a few cells needed for me to quickly update VF_WISESIZE_v#.fits, where # is some integer value representing the table version, using the most recent MAGPHYS and PHOT catalogs. 

In [9]:
import numpy as np
from astropy.table import Table
import os
homedir = os.getenv("HOME")

In [17]:
magphys=Table.read(homedir+'/Desktop/v2-20220820/vf_v2_magphys_legacyExt_final.fits')
phot = Table.read(homedir+'/Desktop/galfit_files/vf_v2_legacy_ephot.fits')
alt_magphys=Table.read(homedir+'/Desktop/galfit_files/vf-altphot.fits')

In [18]:
vfw = Table.read(homedir+'/Desktop/galfit_files/VF_WISESIZE_v2.fits')

### updating SFR, Mstar, sSFR, and subsample flags

In [41]:
sfr_flag = np.zeros(len(vfw),dtype=bool)
mstar_flag = np.zeros(len(vfw),dtype=bool)
ssfr_flag = np.zeros(len(vfw),dtype=bool)
'''
for i in range(len(vfw)):
    if (maskbit_flag[i]):
        sfr_flag[i] = True if ((magphys['logSFR_med'][i]>-1.32) & (magphys['magphysFlag'][i])) else False
        mstar_flag[i] = True if ((magphys['logMstar_med'][i]>8.26) & (magphys['magphysFlag'][i])) else False
        ssfr_flag[i] = True if ((ssfr[i]>-11.5) & (magphys['magphysFlag'][i])) else False
    elif (~maskbit_flag[i]) & (good_flag[i]):
        sfr_flag[i] = True if ((alt_sfr[i]>-1.32) & (magphys['magphysFlag'][i])) else False
        mstar_flag[i] = True if ((alt_mstar[i]>8.26) & (magphys['magphysFlag'][i])) else False
        ssfr_flag[i] = True if ((alt_ssfr[i]>-11.5) & (magphys['magphysFlag'][i])) else False
    else:
        sfr_flag[i] = False
        mstar_flag[i] = False
        ssfr_flag[i] = False
'''
for i in range(len(vfw)):
    sfr_flag[i] = True if (alt_magphys['combined_logSFR_med'][i]>-1.014) else False
    mstar_flag[i] = True if (alt_magphys['combined_logMstar_med'][i]>9.3) else False
    ssfr_flag[i] = True if ((alt_magphys['combined_logSFR_med'][i] - alt_magphys['combined_logMstar_med'][i])>-11.5) else False

In [42]:
vfw['SFRflag'] = sfr_flag
vfw['massflag'] = mstar_flag
vfw['sSFR_flag'] = ssfr_flag

In [43]:
subsampleflag = (vfw['massflag']) & (vfw['SNRflag']) & (vfw['t_flag']) & (vfw['SFRflag']) & (vfw['sSFR_flag'])

In [44]:
vfw['subsample_flag'] = subsampleflag

In [45]:
vfw.write(homedir+'/Desktop/galfit_files/VF_WISESIZE_v2.fits',overwrite=True)

### updating group flags using PHOT catalog from Dr. Moustakas

In [27]:
groupnames = np.zeros(len(vfw),dtype='|S33')
ncomps=np.zeros(len(vfw),dtype=int)
primary_flags = np.zeros(len(vfw),dtype=bool)
group_flags = np.zeros(len(vfw),dtype=bool)
#groupIDs = np.zeros(len(vfw),dtype=int)

In [ ]:
for i in range(len(vfw)):
    if vfw['VFID'][i] in phot['VFID']:
        ncomp = phot[(phot['VFID']==vfw['VFID'][i])]['GROUP_MULT'].data[0]
        groupname = phot[(phot['VFID']==vfw['VFID'][i])]['GROUP_NAME'].data[0]
        primary_flag = phot[(phot['VFID']==vfw['VFID'][i])]['GROUP_PRIMARY'].data[0]
        
    else:
        groupname = 'N/A'
        ncomp=1
        group_flag=False
        primary_flag=False
        
    ncomps[i] = ncomp
    primary_flags[i] = primary_flag
    groupnames[i] = groupname
    print(i)

group_flag = ncomps>1

In [31]:
vfw['primaryGroup_flag'] = primary_flags
vfw['group_name'] = groupnames
vfw['ncomp'] = ncomps
vfw['group_flag'] = group_flags

### (optional) AGN